# 📗 FILE 2-C – CNN & Callbacks

## 🎯 Mục tiêu

Sau bài này bạn sẽ hiểu:
- **CNN** (Convolutional Neural Networks) là gì
- **Conv2D & Pooling** layers
- Build CNN cho **Image Classification**
- **Callbacks** - EarlyStopping, ModelCheckpoint
- Best practices cho Computer Vision

---

## 📌 Tại sao CNN?

**Vấn đề với Dense layers cho images:**
```
Image 28×28 = 784 pixels
Dense(128) → 784 × 128 = 100,352 parameters!
```
- ❌ Quá nhiều parameters
- ❌ Không tận dụng spatial structure
- ❌ Không translation invariant

**CNN giải quyết:**
- ✅ Parameter sharing
- ✅ Local connectivity
- ✅ Translation invariance
- ✅ Hierarchical features

---

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

print(f"TensorFlow version: {tf.__version__}")

---

## 1️⃣ Convolution - Cơ chế cốt lõi

### 🔹 Convolution là gì?

**Idea:** Slide a **filter** (kernel) over image to detect patterns

```
Image:           Filter:        Output:
[1 2 3]          [1 0]          [1×1+2×0+4×1+5×0] = 5
[4 5 6]    *     [1 0]    =     [2×1+3×0+5×1+6×0] = 7
[7 8 9]                         ...
```

**Key concepts:**
- **Filter/Kernel**: Small matrix (e.g., 3×3)
- **Stride**: Step size
- **Padding**: Add borders to maintain size
- **Feature map**: Output of convolution

---

In [ ]:
# Visualize convolution
# Create simple image
image = np.array([
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
], dtype=np.float32)

# Filters
filter_vertical = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1]
], dtype=np.float32)

filter_horizontal = np.array([
    [-1, -1, -1],
    [ 0,  0,  0],
    [ 1,  1,  1]
], dtype=np.float32)

# Reshape for TensorFlow (batch, height, width, channels)
image_tf = image.reshape(1, 5, 5, 1)
filter_v_tf = filter_vertical.reshape(3, 3, 1, 1)
filter_h_tf = filter_horizontal.reshape(3, 3, 1, 1)

# Apply convolution
output_v = tf.nn.conv2d(image_tf, filter_v_tf, strides=1, padding='VALID')
output_h = tf.nn.conv2d(image_tf, filter_h_tf, strides=1, padding='VALID')

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].imshow(image, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(output_v[0, :, :, 0], cmap='gray')
axes[1].set_title('Vertical Edge Detection')
axes[1].axis('off')

axes[2].imshow(output_h[0, :, :, 0], cmap='gray')
axes[2].set_title('Horizontal Edge Detection')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("Convolution detected edges in the image!")

---

## 2️⃣ Conv2D Layer

### 🔹 Syntax

In [ ]:
# Basic Conv2D
conv_layer = tf.keras.layers.Conv2D(
    filters=32,              # Số filters (output channels)
    kernel_size=(3, 3),      # Kích thước filter
    strides=(1, 1),          # Stride (default: 1)
    padding='same',          # 'same' or 'valid'
    activation='relu',       # Activation function
    input_shape=(28, 28, 1)  # (height, width, channels)
)

print(conv_layer)

### 🔹 Padding: 'same' vs 'valid'

**'valid' (no padding):**
```
Input:  (28, 28)
Filter: (3, 3)
Output: (26, 26)  # Shrinks!
```

**'same' (with padding):**
```
Input:  (28, 28)
Filter: (3, 3)
Output: (28, 28)  # Same size!
```

**Best practice:** Dùng `'same'` để maintain spatial dimensions

---

In [ ]:
# Demo padding effect
input_tensor = tf.random.normal([1, 28, 28, 1])

# Valid padding
conv_valid = tf.keras.layers.Conv2D(32, (3, 3), padding='valid')
output_valid = conv_valid(input_tensor)

# Same padding
conv_same = tf.keras.layers.Conv2D(32, (3, 3), padding='same')
output_same = conv_same(input_tensor)

print(f"Input shape:  {input_tensor.shape}")
print(f"Valid output: {output_valid.shape}")
print(f"Same output:  {output_same.shape}")

---

## 3️⃣ Pooling Layers

### 🔹 Tại sao cần Pooling?

**Goals:**
- Reduce spatial dimensions → fewer parameters
- Add translation invariance
- Prevent overfitting

### 🔹 MaxPooling vs AveragePooling

In [ ]:
# Example
feature_map = np.array([
    [1, 3, 2, 4],
    [5, 6, 7, 8],
    [3, 2, 1, 0],
    [1, 2, 3, 4]
], dtype=np.float32)

# Reshape for TensorFlow
feature_map_tf = feature_map.reshape(1, 4, 4, 1)

# MaxPooling (2×2)
maxpool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
output_max = maxpool(feature_map_tf)

# AveragePooling (2×2)
avgpool = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))
output_avg = avgpool(feature_map_tf)

print("Original:")
print(feature_map)
print("\nMaxPooling (2×2):")
print(output_max[0, :, :, 0].numpy())
print("\nAveragePooling (2×2):")
print(output_avg[0, :, :, 0].numpy())

print("\nMaxPooling: Picks maximum value → retains strong features")
print("AveragePooling: Takes average → smoother")

### 🔹 Khi nào dùng?

- **MaxPooling**: Default choice, tốt cho edge/texture detection
- **AveragePooling**: Smoother, ít dùng trong hidden layers
- **GlobalAveragePooling**: Thay thế Flatten ở cuối network

---

---

## 4️⃣ CNN Architecture - Typical Pattern

### 🔹 Standard CNN pattern

```
Input Image
    ↓
[Conv2D → ReLU → MaxPool] × N  # Feature extraction
    ↓
Flatten
    ↓
[Dense → ReLU → Dropout] × M   # Classification
    ↓
Output (Softmax)
```

**Key principles:**
- Filters increase: 32 → 64 → 128
- Spatial size decreases: 28×28 → 14×14 → 7×7
- Add Dropout before Dense layers

---

---

## 5️⃣ Build CNN for MNIST

### 🔹 Load & prepare data

In [ ]:
# Load MNIST
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

# Visualize samples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_train[i], cmap='gray')
    ax.set_title(f'Label: {y_train[i]}')
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Preprocess
# 1. Reshape to add channel dimension
X_train = X_train.reshape(-1, 28, 28, 1).astype(np.float32)
X_test = X_test.reshape(-1, 28, 28, 1).astype(np.float32)

# 2. Normalize to [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0

# 3. Labels (already 0-9, no need to one-hot for sparse_categorical_crossentropy)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Value range: [{X_train.min():.2f}, {X_train.max():.2f}]")

### 🔹 Build CNN model

In [ ]:
# Simple CNN
model_cnn = tf.keras.Sequential([
    # Block 1
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Block 2
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Block 3
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    
    # Classifier
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
], name='simple_cnn')

print(model_cnn.summary())

In [ ]:
# Compile
model_cnn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Model compiled!")

---

## 6️⃣ Callbacks - Smart Training Control

### 🔹 Callbacks là gì?

**Callbacks** = Functions được gọi tại các điểm trong training

**Use cases:**
- Stop training early (EarlyStopping)
- Save best model (ModelCheckpoint)
- Adjust learning rate (ReduceLROnPlateau)
- Log to TensorBoard
- Custom logic

---

### 🔹 1. EarlyStopping

**Problem:** Không biết train bao nhiêu epochs

**Solution:** Stop khi val_loss không improve

**Parameters:**
- `monitor`: Metric to monitor (e.g., 'val_loss')
- `patience`: Số epochs chờ trước khi stop
- `restore_best_weights`: Restore weights từ best epoch

---

In [ ]:
# EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',         # Theo dõi validation loss
    patience=5,                 # Chờ 5 epochs
    restore_best_weights=True,  # Restore best model
    verbose=1
)

print("EarlyStopping configured:")
print(f"  - Monitor: val_loss")
print(f"  - Patience: 5 epochs")
print(f"  - Will restore best weights")

### 🔹 2. ModelCheckpoint

**Problem:** Muốn save best model during training

**Solution:** Auto save khi val_loss improve

**Parameters:**
- `filepath`: Nơi save model
- `monitor`: Metric to monitor
- `save_best_only`: Chỉ save khi improve
- `save_weights_only`: Save weights only (faster)

---

In [ ]:
# ModelCheckpoint callback
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='/tmp/best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

print("ModelCheckpoint configured:")
print(f"  - Filepath: /tmp/best_model.keras")
print(f"  - Monitor: val_accuracy")
print(f"  - Save best only: True")

### 🔹 3. ReduceLROnPlateau

**Problem:** Learning rate không tối ưu

**Solution:** Giảm LR khi val_loss plateau

---

In [ ]:
# ReduceLROnPlateau callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,          # LR mới = LR cũ × 0.5
    patience=3,          # Chờ 3 epochs
    min_lr=1e-7,         # Minimum LR
    verbose=1
)

print("ReduceLROnPlateau configured:")
print(f"  - Factor: 0.5 (halve LR)")
print(f"  - Patience: 3 epochs")

### 🔹 Train with Callbacks

In [ ]:
# Combine callbacks
callbacks = [
    early_stopping,
    checkpoint,
    reduce_lr
]

# Train
print("Training with callbacks...")
history = model_cnn.fit(
    X_train, y_train,
    epochs=30,                    # Set high, EarlyStopping will stop
    batch_size=128,
    validation_split=0.1,
    callbacks=callbacks,          # Add callbacks!
    verbose=1
)

print("\nTraining completed!")

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training & Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history.history['accuracy'], label='Train Acc', linewidth=2)
axes[1].plot(history.history['val_accuracy'], label='Val Acc', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training & Validation Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Training stopped at epoch: {len(history.history['loss'])}")
print(f"Best val_accuracy: {max(history.history['val_accuracy']):.4f}")

In [ ]:
# Evaluate on test set
test_loss, test_acc = model_cnn.evaluate(X_test, y_test, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
# Load best model
best_model = tf.keras.models.load_model('/tmp/best_model.keras')
best_test_loss, best_test_acc = best_model.evaluate(X_test, y_test, verbose=0)

print(f"Best Model Test Accuracy: {best_test_acc:.4f}")

---

## 7️⃣ Visualize CNN Predictions

In [ ]:
# Make predictions
predictions = model_cnn.predict(X_test[:20])
predicted_classes = np.argmax(predictions, axis=1)

# Visualize
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_test[i].reshape(28, 28), cmap='gray')
    
    # Color: green if correct, red if wrong
    color = 'green' if predicted_classes[i] == y_test[i] else 'red'
    ax.set_title(f'True: {y_test[i]}, Pred: {predicted_classes[i]}', color=color)
    ax.axis('off')

plt.tight_layout()
plt.show()

correct = np.sum(predicted_classes == y_test[:20])
print(f"Correct predictions: {correct}/20")

---

## 8️⃣ Data Augmentation

### 🔹 Tại sao cần augmentation?

**Problem:** Limited training data → overfitting

**Solution:** Tạo variations của training images

**Techniques:**
- Rotation
- Flip
- Zoom
- Shift
- Brightness

---

In [ ]:
# Data augmentation layer (TF 2.x style)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.1),       # ±10% rotation
    tf.keras.layers.RandomZoom(0.1),           # ±10% zoom
    tf.keras.layers.RandomTranslation(0.1, 0.1)  # ±10% shift
])

# Visualize augmentation
sample_image = X_train[0:1]  # Take first image

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i, ax in enumerate(axes.flat):
    augmented = data_augmentation(sample_image, training=True)
    ax.imshow(augmented[0, :, :, 0], cmap='gray')
    ax.set_title(f'Augmented {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# CNN with data augmentation
model_with_aug = tf.keras.Sequential([
    # Data augmentation (only during training)
    data_augmentation,
    
    # CNN layers
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    
    # Classifier
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

model_with_aug.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Model with augmentation ready!")

---

## 9️⃣ Best Practices Summary

### ✅ CNN Architecture

```python
# Standard pattern
model = Sequential([
    # Feature extraction
    Conv2D(32, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    
    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    
    # More layers...
    
    # Classification head
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])
```

### ✅ Training Setup

```python
# Always use callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]

# Train
model.fit(X_train, y_train, 
          validation_split=0.1,
          callbacks=callbacks,
          epochs=100)  # Set high, callbacks will handle it
```

### ✅ Data Preparation

1. Normalize: `X / 255.0`
2. Reshape: Add channel dimension
3. Augmentation: For small datasets
4. Validation split: 10-20%

---

---

## 🔟 Exercises

### 📝 Exercise 1: Deeper CNN

Build deeper CNN:
- 4 conv blocks instead of 3
- Filters: 32 → 64 → 128 → 128
- Compare with simple CNN

In [ ]:
# YOUR CODE HERE
# TODO: Build deeper CNN

### 📝 Exercise 2: Custom Callback

Implement custom callback:
- Print learning rate at end of each epoch
- Save model every 5 epochs

In [ ]:
# YOUR CODE HERE
class CustomCallback(tf.keras.callbacks.Callback):
    # TODO: Implement
    pass

### 📝 Exercise 3: Fashion MNIST

Apply CNN to Fashion MNIST:
- Load fashion_mnist dataset
- Build CNN (similar to MNIST)
- Use callbacks
- Achieve >90% test accuracy

In [ ]:
# YOUR CODE HERE
# TODO: Complete Fashion MNIST classification

---

## 🎯 Tóm tắt

### ✅ Đã học

1. **CNN Basics**: Convolution, Pooling
2. **Conv2D & MaxPooling2D** layers
3. **CNN Architecture** patterns
4. **Image Classification** with MNIST
5. **Callbacks**: EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
6. **Data Augmentation**

### 🎓 Key Takeaways

**CNN Pattern:**
```
Input → [Conv→ReLU→Pool]×N → Flatten → [Dense→Dropout]×M → Output
```

**Always use:**
- EarlyStopping (avoid overfitting)
- ModelCheckpoint (save best model)
- Data augmentation (if dataset small)

### 📚 What's Next?

**PHẦN 3 - ADVANCED** (nếu có):
- Transfer Learning
- Advanced CNN architectures
- Multi-GPU training

---

## 🎉 Hoàn thành PHẦN 2!

Bạn đã nắm được:
- ✅ Data pipeline (tf.data)
- ✅ Optimizers & Regularization
- ✅ CNN & Callbacks

**Ready for production!** 🚀

---

## 📖 References

- [Keras CNN Guide](https://www.tensorflow.org/tutorials/images/cnn)
- [Keras Callbacks](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks)
- [Data Augmentation](https://www.tensorflow.org/tutorials/images/data_augmentation)

---

**Chúc bạn học tốt! 🚀**